# Library importing

In [6]:
import cv2
import numpy as np
from numpy.core.numeric import True_
from numpy.lib.arraysetops import unique
import pandas as pd
# import glob
import os
from pandas import json_normalize
from os import getcwd, path
from yaml import SafeLoader, load
import datetime
import matplotlib.pyplot as plt
import yaml
import shutil
import json

from video_utils_virat import VideoUtils
from video_utils_virat import YoloUtils

%matplotlib inline
cv2.__version__


'4.5.4'

# Setting paths to location of annotations
A "processed" directory will be created containing a folder with the video name. All results will be placed in this folder.

In [7]:
# setup
dataset_dir_path = '../datasets/VIRAT/'
image_ext = '.jpg'
video_max_frames = 2000

#video
video_ext = '.mp4'
video_name = 'VIRAT_S_050000_07_001014_001126'
video_name_orig = video_name + video_ext
video_dest_path = './processed/' +  video_name + '/'
video_src_path = dataset_dir_path + 'Videos/Ground/'

using_yml = True

# annotations
saved_csv = video_dest_path + 'df_bbox.csv'


yml_video_name = 'gt_'
yolo_video_name = 'yolo_'

# annotations_path = dataset_dir_path + 'viratannotations/train/' + video_name +'/'
annotations_path = dataset_dir_path + 'viratannotations/validate/' + video_name +'/'
ann_activities_file = annotations_path + video_name + '.activities.yml'
ann_geom_file = annotations_path + video_name + '.geom.yml'
ann_regions_file = annotations_path + video_name + '.regions.yml'
ann_types_file = annotations_path + video_name + '.types.yml'


gt_video_name = yml_video_name + video_name + '.mp4'#video_ext
yolo_video_name = yolo_video_name + video_name + '.mp4'#video_ext

## Find ground truth annotation types
Create a dataframe containing types from the ground truth annotations

In [8]:
# Create directory to store new video
if not os.path.exists(video_dest_path):
    os.makedirs(video_dest_path)

if not os.path.exists(saved_csv):
    with open(ann_types_file) as yaml_file:
        yaml_contents = load(yaml_file, Loader=SafeLoader)
    yaml_df = json_normalize(yaml_contents)
    yaml_df
    for col in yaml_df.columns:
        type_name = col.split('.')[-1]
        if not (type_name == 'id1'):
            yaml_df.loc[yaml_df[col] == 1, col] = type_name
    
    yaml_df = yaml_df[yaml_df['types.id1'].notna()].reset_index().dropna(axis=1, how='all')  
    type_df = yaml_df.ffill(axis=1).iloc[:,-1].to_frame(name='category')
    type_df.insert(0, "id", yaml_df['types.id1'])
    type_df




# Creating annotation dataframe
Fill a dataframe which contains all the annotation data (bounding boxes, types, IDs)
For faster load, first check if a csv has been pre created for this video annotation. Otherwise create the dataframe from yaml file and save it for next time. 

In [9]:
# using annotations:
print("Loading annotations...")
def add_category_type(row):
  id = row['object_id']
  val = type_df.loc[type_df['id'] == id, 'category'].iloc[0]
  return val


if os.path.exists(saved_csv):
  df_bbox = pd.read_csv(saved_csv)
else:
  with open(ann_geom_file) as yaml_file:
      yaml_contents = load(yaml_file, Loader=SafeLoader)
  yaml_df = json_normalize(yaml_contents)

  df_bbox = yaml_df[['geom.id1','geom.ts0','geom.ts1','geom.g0']].dropna().reset_index()
  df_bbox.rename(columns={'geom.id1': 'object_id', 'geom.ts0': 'frame_id','geom.ts1': 'time_sec', 'geom.g0': 'bbox'}, inplace=True)
  df_bbox['bbox'] = df_bbox['bbox'].str.split()
  df_tmp = pd.DataFrame(df_bbox['bbox'].to_list(), columns = ['bb_left', 'bb_top', 'bb_right', 'bb_bottom'])
  df_bbox = pd.concat([df_bbox, df_tmp], axis=1).drop(columns=['bbox'])

  df_bbox['category'] = df_bbox.apply(lambda row: add_category_type(row), axis=1) 
  df_bbox.drop(columns=['index'], axis=1, inplace=True)
  # df_bbox.set_index['index'] 
  df_bbox.to_csv(saved_csv, index = False)
    

df_bbox.head()

Loading annotations...


,object_id,frame_id,time_sec,bb_left,bb_top,bb_right,bb_bottom,category
0,0.0,0.0,0.000000,485,743,653,914,Vehicle
1,0.0,1.0,0.033333,489,748,657,919,Vehicle
2,0.0,2.0,0.066667,488,747,656,918,Vehicle
3,0.0,3.0,0.100000,488,747,656,918,Vehicle
4,0.0,4.0,0.133333,488,747,656,918,Vehicle


# Add velocities

In [10]:
# Add velocities for the video

vidcap = cv2.VideoCapture(video_src_path + video_name_orig)
fps = 30
scale = 1/15
if vidcap.isOpened():
    fps = vidcap.get(cv2.CAP_PROP_FPS)  
    print(fps)
    # scale = #TBD


def find_mid(a, b):
  c = (a.astype(float) + b.astype(float))/2
  return c

def find_delta(x):
  dx = x.diff()
  dx[0] = np.nan
  dx.fillna(method='backfill', inplace=True)
  return dx

def find_vel(vx,vy):

  vel = np.sqrt(vx * vx + vy * vy).round(2)
  return vel#, vx, vy

for id in df_bbox['object_id'].unique():
  mask = (df_bbox['object_id']==id)
  sub_df = df_bbox[mask]
  x  = find_mid(sub_df['bb_top'], sub_df['bb_bottom'])
  y  = find_mid(sub_df['bb_left'], sub_df['bb_right'])
  dx = find_delta(x)
  dy = find_delta(y)
  vx = (dx * fps * scale)
  vy = (dy * fps * scale)

  df_bbox.loc[mask, 'x']  = x
  df_bbox.loc[mask, 'y']  = y
  df_bbox.loc[mask, 'dx'] = dx
  df_bbox.loc[mask, 'dy'] = dy
  df_bbox.loc[mask, 'vx'] = (dx * fps * scale)
  df_bbox.loc[mask, 'vy'] = (dy * fps * scale)
  df_bbox.loc[mask, 'vel']  = np.sqrt(vx * vx + vy * vy).round(2)

df_bbox.head(50)

29.97002997002997


,object_id,frame_id,time_sec,bb_left,bb_top,bb_right,bb_bottom,category,x,y,dx,dy,vx,vy,vel
0,0.0,0.0,0.000000,485,743,653,914,Vehicle,828.5,569.0,5.0,4.0,9.990010,7.992008,12.79
1,0.0,1.0,0.033333,489,748,657,919,Vehicle,833.5,573.0,5.0,4.0,9.990010,7.992008,12.79
2,0.0,2.0,0.066667,488,747,656,918,Vehicle,832.5,572.0,-1.0,-1.0,-1.998002,-1.998002,2.83
3,0.0,3.0,0.100000,488,747,656,918,Vehicle,832.5,572.0,0.0,0.0,0.000000,0.000000,0.00
4,0.0,4.0,0.133333,488,747,656,918,Vehicle,832.5,572.0,0.0,0.0,0.000000,0.000000,0.00
5,0.0,5.0,0.166667,488,746,656,917,Vehicle,831.5,572.0,-1.0,0.0,-1.998002,0.000000,2.00
6,0.0,6.0,0.200000,488,746,656,917,Vehicle,831.5,572.0,0.0,0.0,0.000000,0.000000,0.00
7,0.0,7.0,0.233333,488,746,656,917,Vehicle,831.5,572.0,0.0,0.0,0.000000,0.000000,0.00
8,0.0,8.0,0.266667,488,745,656,916,Vehicle,830.5,572.0,-1.0,0.0,-1.998002,0.000000,2.00
9,0.0,9.0,0.300000,488,745,656,916,Vehicle,830.5,572.0,0.0,0.0,0.000000,0.000000,0.00
